# AI-Generated Image Detection using CNN

This notebook implements a Convolutional Neural Network (CNN) to classify images as **Real** or **AI-Generated (Fake)**. 

## Overview

1. **Model Architecture**: A custom CNN with 3 convolutional blocks for feature extraction
2. **Data Pipeline**: Loading, preprocessing, and augmentation of images
3. **Training Loop**: Complete training with validation and model checkpointing
4. **Evaluation**: Performance metrics including Accuracy, Precision, Recall, and F1-Score

## Dataset Structure

The dataset should be organized in the following folder structure:
```
archive/
├── FAKE/    # AI-generated images
└── REAL/    # Real photographs
```

## 1. Import Libraries

Import all necessary libraries for deep learning, data handling, and evaluation metrics.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

## 2. Configuration

Define hyperparameters and settings for the model training. These can be adjusted to experiment with different configurations.

In [ ]:
# Configuration
DATA_DIR = 'archive'
BATCH_SIZE = 32
IMG_SIZE = 128
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Specific device selection: Use GPU (cuda) if available for faster training, otherwise fallback to CPU.
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

## 3. Model Architecture

The CNN architecture consists of:
- **3 Convolutional Blocks**: Each block contains Conv2D → BatchNorm → ReLU → MaxPool
- **Fully Connected Layers**: For classification after feature extraction
- **Dropout**: To prevent overfitting

### Architecture Details

| Layer | Input Size | Output Size | Description |
|-------|------------|-------------|-------------|
| Conv1 | 128×128×3 | 128×128×32 | Extract low-level features |
| Pool1 | 128×128×32 | 64×64×32 | Reduce spatial dimensions |
| Conv2 | 64×64×32 | 64×64×64 | Extract mid-level features |
| Pool2 | 64×64×64 | 32×32×64 | Reduce spatial dimensions |
| Conv3 | 32×32×64 | 32×32×128 | Extract high-level features |
| Pool3 | 32×32×128 | 16×16×128 | Reduce spatial dimensions |
| FC1 | 32768 | 512 | Classification layer |
| FC2 | 512 | 1 | Output (logit) |

In [ ]:
class AIImageDetectorCNN(nn.Module):
    """
    A Convolutional Neural Network (CNN) for binary image classification (Real vs Fake).
    """
    def __init__(self):
        super(AIImageDetectorCNN, self).__init__()
        
        # Convolutional Layers (Feature Extraction)
        # We assume input images are resized to 128x128 pixels with 3 color channels (RGB).
        # Input Shape: [Batch_Size, 3, 128, 128]
        
        # Layer 1: 
        # Conv2d: Extracts low-level features (edges, colors).
        # out_channels=32: Creates 32 different feature maps.
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # BatchNorm2d: Normalizes the output of the convolution. 
        # Helps training stability and speed by keeping activation distributions consistent.
        self.bn1 = nn.BatchNorm2d(32)
        
        # Layer 2:
        # Increases depth to 64 channels to capture more complex textures/patterns.
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Layer 3:
        # Increases depth to 128 channels for high-level feature abstraction.
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Pooling Layer: 
        # MaxPool2d reduces spatial dimensions (height/width) by half (stride=2).
        # This reduces computation and makes the model translation invariant (robust to position shifts).
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Dropout: 
        # Randomly zeros out 50% of neurons during training.
        # Prevents overfitting by forcing the network to learn redundant representations.
        self.dropout = nn.Dropout(0.5)
        
        # Fully Connected Layers (Classification)
        # Calculate Flattened Input Size:
        # Original Image: 128x128
        # After Pool 1: 64x64
        # After Pool 2: 32x32
        # After Pool 3: 16x16
        # Final Tensor Shape before flattening: [Batch_Size, 128 (channels), 16, 16]
        # Flattened Vector Size = 128 * 16 * 16 = 32768
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        
        # Output Layer: 
        # Maps the 512 features to a single value (logit).
        # A positive value suggests one class (e.g., Real), negative suggests the other (Fake).
        self.fc2 = nn.Linear(512, 1) 

    def forward(self, x):
        """
        Defines the forward pass (data flow) of the network.
        Args:
            x: Input batch of images.
        Returns:
            x: Unnormalized output scores (logits).
        """
        
        # Block 1: Conv -> BN -> ReLU -> Pool
        # ReLU (Rectified Linear Unit) introduces non-linearity, allowing the model to learn complex functions.
        # without ReLU, the model would just be a linear regression.
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Block 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Flattening:
        # Reshapes the 3D feature maps (Channels, Height, Width) into a 1D vector
        # so it can be fed into the Fully Connected (Dense) layers.
        # x.size(0) preserves the batch size. -1 infers the remaining dimension size.
        x = x.view(x.size(0), -1)
        
        # Classification Head:
        # FC1 -> ReLU -> Dropout
        x = self.dropout(F.relu(self.fc1(x)))
        
        # Final Output (Logit)
        # We do NOT apply Sigmoid here because we use BCEWithLogitsLoss during training,
        # which applies Sigmoid internally for better numerical stability.
        x = self.fc2(x)
        
        return x

## 4. Data Loading and Preprocessing

The data pipeline includes:
- **Resize**: All images are resized to 128×128 pixels
- **ToTensor**: Converts PIL images (0-255) to PyTorch tensors (0-1)
- **Normalize**: Standardizes pixel values using ImageNet statistics for better convergence

The dataset is split into **80% training** and **20% validation**.

In [ ]:
def get_data_loaders(data_dir, batch_size, img_size):
    """
    Prepares the training and validation data loaders.
    
    Args:
        data_dir (str): Path to the dataset directory.
        batch_size (int): Number of images per batch.
        img_size (int): Target size to resize images.
        
    Returns:
        train_loader: DataLoader for training data.
        val_loader: DataLoader for validation data.
        class_to_idx: Dictionary mapping class names to indices.
    """
    # Data augmentation and normalization for training
    # - Resize: Ensures all images are the same size for the neural network input.
    # - ToTensor: Converts PIL images (0-255) to PyTorch tensors (0-1).
    # - Normalize: Standardizes pixel values to resemble ImageNet statistics (mean ~0, std ~1).
    #   This helps the model converge (learn) faster and reach a stable solution.
    #   Values (0.485, ...) are standard constants for pre-trained models/general natural images gathered from the internet.
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load dataset from folder structure (folder name = class label)
    try:
        full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    except FileNotFoundError:
        print(f"Error: Data directory '{data_dir}' not found.")
        return None, None, None

    # Split into train and validation (80/20)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    print(f"Dataset loaded. Classes: {full_dataset.classes}")
    print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

    # DataLoader handles batching, shuffling, and parallel data loading (num_workers).
    # Shuffle=True for training prevents the model from learning order-based patterns.
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, full_dataset.class_to_idx

## 5. Evaluation Function

This function calculates key classification metrics:
- **Accuracy**: Overall correct predictions
- **Precision**: True positives / (True positives + False positives)
- **Recall**: True positives / (True positives + False negatives)
- **F1-Score**: Harmonic mean of precision and recall

In [ ]:
def evaluate_model(model, loader, device):
    """
    Evaluates the model's performance on a given dataset (validation/test).
    """
    # Set model to evaluation mode.
    # Disables Dropout and switches BatchNorm to use running statistics.
    model.eval()
    
    all_preds = []
    all_labels = []
    
    # torch.no_grad() disables gradient calculation.
    # We don't need gradients for evaluation, and this saves memory and computation.
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()
            
            # Move results back to CPU and convert to numpy for scikit-learn metrics
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    # Calculate metrics using scikit-learn
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    
    return acc, prec, rec, f1

## 6. Training Function

The training loop implements:
- **Forward pass**: Compute predictions
- **Loss calculation**: Using Binary Cross-Entropy with Logits
- **Backward pass**: Compute gradients via backpropagation
- **Optimizer step**: Update model weights
- **Model checkpointing**: Save the best model based on validation accuracy

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    """
    Executes the training loop.
    
    Args:
        model: The neural network model.
        train_loader: DataLoader for training data.
        val_loader: DataLoader for validation data.
        criterion: Loss function (BCEWithLogitsLoss).
        optimizer: Optimization algorithm (Adam).
        num_epochs: Number of times to iterate over the entire dataset.
        device: 'cuda' or 'cpu'.
    """
    best_acc = 0.0
    
    print(f"Starting training on {device}...")
    
    for epoch in range(num_epochs):
        # Set model to training mode. 
        # This enables layers like Dropout and BatchNorm that behave differently during training vs inference.
        model.train()
        
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # tqdm creates a visual progress bar for the epoch loop (wraps around the DataLoader)
        loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        
        for inputs, labels in loop:
            # Move data to the configured device (GPU/CPU) for computation
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients.
            # PyTorch accumulates gradients by default. We must clear them before the backward pass
            # of this batch, otherwise gradients from previous batches would mix in.
            optimizer.zero_grad()
            
            # Forward Pass: Compute predicted outputs by passing inputs to the model
            outputs = model(inputs)
            
            # Prepare labels for Loss Calculation
            # BCEWithLogitsLoss expects labels to be Float tensors.
            # .unsqueeze(1) reshapes labels from [batch_size] to [batch_size, 1] to match the shape of the model output.
            labels = labels.float().unsqueeze(1) 
            
            # Calculate Loss: Determine how wrong the model's predictions are compared to actual labels
            loss = criterion(outputs, labels)
            
            # Backward Pass (Backpropagation):
            # Calculates the gradient of the loss with respect to model parameters.
            # It figures out "direction" and "magnitude" to adjust weights to reduce error.
            loss.backward()
            
            # Optimizer Step:
            # Updates the model's weights based on the computed gradients.
            optimizer.step()
            
            # Statistics & Metrics Tracking:
            
            # Add current batch loss to running total (loss.item() extracts the scalar value)
            running_loss += loss.item()
            
            # Convert logits (raw output) to probabilities using Sigmoid (0 to 1 range)
            probs = torch.sigmoid(outputs)
            
            # Threshold probabilities to binary predictions (0 or 1)
            preds = (probs > 0.5).float()
            
            # Count correct predictions
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
            
            # Update progress bar with current batch loss
            loop.set_postfix(loss=loss.item())

        # Calculate average loss and accuracy for the epoch
        train_acc = correct_train / total_train
        avg_loss = running_loss / len(train_loader)
        
        # Validation phase: Evaluate model on unseen data
        val_acc, val_precision, val_recall, val_f1 = evaluate_model(model, val_loader, device)
        
        print(f"Epoch {epoch+1}/{num_epochs} -> "
              f"Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
        
        # Save the best model based on validation accuracy.
        # This ensures we keep the version of the model that generalized best, 
        # not necessarily the one from the very last epoch (which might be overfitting).
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved best model.")

    print("Training complete.")

## 7. Load and Prepare Data

Load the dataset and create data loaders for training and validation.

In [ ]:
# Load data
train_loader, val_loader, class_mapping = get_data_loaders(DATA_DIR, BATCH_SIZE, IMG_SIZE)

# Display class mapping
print(f"Class Mapping: {class_mapping}")  # {'FAKE': 0, 'REAL': 1}

## 8. Initialize Model, Loss Function, and Optimizer

- **Model**: Custom CNN architecture
- **Loss Function**: `BCEWithLogitsLoss` - combines Sigmoid + Binary Cross-Entropy for numerical stability
- **Optimizer**: `Adam` - Adaptive Moment Estimation, generally converges faster than SGD

In [ ]:
# Initialize the model and move to device
model = AIImageDetectorCNN().to(DEVICE)

# BCEWithLogitsLoss combines Sigmoid layer + BCELoss in one class.
# This is numerically more stable than using a plain Sigmoid followed by BCELoss.
criterion = nn.BCEWithLogitsLoss()

# Adam optimizer: Adaptive Moment Estimation.
# Generally performs better and converges faster than SGD for many deep learning tasks.
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Display model summary
print(f"Model moved to: {DEVICE}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## 9. Train the Model

Run the training loop. The best model (based on validation accuracy) will be automatically saved to `best_model.pth`.

In [ ]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE)

## 10. Final Evaluation

Load the best saved model and evaluate its performance on the validation set.

In [ ]:
# Load the best weights saved during training (not necessarily the last epoch's weights)
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on validation set
print("--- Final Evaluation on Validation Set ---")
acc, prec, rec, f1 = evaluate_model(model, val_loader, DEVICE)

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")